<a href="https://colab.research.google.com/github/cicl-iscl/LeWiDi_SemEval2023/blob/main/Notebooks/GaussianProcess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Notes

ConvAbuse is just the dictionary as a string lol

In [ ]:
!pip install transformers
!pip install datasets
!pip install evaluate
!pip install wandb
!pip install torchsampler
!pip install gpytorch

# Set up wandb

In [2]:
import wandb
# config = dict (
#   architecture = "bert_arabic_english",
#   epochs = 1,
#   batch_size = 4
# )

# run = wandb.init(
#   project="capture-disagreement",
#   notes="test_run",
#   #name = "test_run",
#   config=config,
#   save_code = True,
#   tags = ["bert_arabic_english", "weighted_loss", "balanced_loss", "only_last_layer"],
#   #magic = True
# )

run = wandb.init(
    project="capture-disagreement",
    save_code = True,
)

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: eolian (capture_disagreement). Use `wandb login --relogin` to force relogin


# Load data

In [3]:
bert_multilingual_last_layer_dataset = run.use_artifact('capture_disagreement/capture-disagreement/bert_multilingual_last_layer_dataset:latest', type='dataset')
bert_multilingual_last_layer_dataset_directory = bert_multilingual_last_layer_dataset.download()

wandb: Downloading large artifact bert_multilingual_last_layer_dataset:latest, 265.92MB. 10431 files... 
wandb:   10431 of 10431 files downloaded.  
Done. 0:0:1.1


In [4]:
import torch
import os

class ModelOutputDataset(torch.utils.data.Dataset):
    def __init__(self, output_dir):
        self.output_dir = output_dir
        self.output_paths = [os.path.join(output_dir, path) for path in os.listdir(output_dir)]
        
    def __len__(self):
        return len(self.output_paths)
    
    def __getitem__(self, index):
        output_path = self.output_paths[index]
        data = torch.load(output_path, map_location=device)
        return data

In [5]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
bert_multilingual_last_layer_dataset = ModelOutputDataset(bert_multilingual_last_layer_dataset_directory)

## Balanced

In [7]:
# import torch
# import os
# from collections import Counter

# # Check if PyTorch version is greater than 1.5.0
# if int(torch.__version__[0]) > 1 or (int(torch.__version__[0]) == 1 and int(torch.__version__[2]) >= 5):
#     from torch.utils.data.sampler import WeightedRandomSampler
# else:
#     # Implement WeightedRandomSampler using custom code
#     class WeightedRandomSampler:
#         def __init__(self, weights, num_samples, replacement=True):
#             self.weights = torch.DoubleTensor(weights)
#             self.num_samples = num_samples
#             self.replacement = replacement

#         def __iter__(self):
#             return iter(torch.multinomial(self.weights, self.num_samples, self.replacement))

#         def __len__(self):
#             return self.num_samples

# class ModelOutputDataset(torch.utils.data.Dataset):
#     def __init__(self, output_dir):
#         self.output_dir = output_dir
#         self.output_paths = [os.path.join(output_dir, path) for path in os.listdir(output_dir)]
        
#     def __len__(self):
#         return len(self.output_paths)
    
#     def __getitem__(self, index):
#         output_path = self.output_paths[index]
#         data = torch.load(output_path, map_location=device)
#         return data

# bert_multilingual_last_layer_dataset = ModelOutputDataset(bert_multilingual_last_layer_dataset_directory)

# # Split the dataset into train and test
# train_set, test_set = torch.utils.data.random_split(bert_multilingual_last_layer_dataset, [0.8, 0.2])
# test_set, eval_set = torch.utils.data.random_split(test_set, [0.5, 0.5])

# # Get the labels for each sample
# train_labels = [data["labels"].cpu().item() for data in train_set]
# eval_labels = [data["labels"] for data in eval_set]
# test_labels = [data["labels"] for data in test_set]

# # Count the number of samples for each class in the train set
# label_counts = Counter(train_labels)

# # Calculate the weight for each sample
# weights = [1.0 / label_counts[label] for label in train_labels]

# # Create a weighted random sampler for the train set
# sampler = WeightedRandomSampler(weights, len(weights))

# # Set up a dataloader for the train set
# train_dataloader = torch.utils.data.DataLoader(train_set, batch_size=256, sampler=sampler)
# eval_dataloader = torch.utils.data.DataLoader(eval_set, batch_size=256, shuffle=False)
# test_dataloader = torch.utils.data.DataLoader(test_set, batch_size=256, shuffle=False)

## Unbalanced

In [8]:
# Split the dataset into train and test
train_set, test_set = torch.utils.data.random_split(bert_multilingual_last_layer_dataset, [0.8, 0.2])
test_set, eval_set = torch.utils.data.random_split(test_set, [0.5, 0.5])


# Set up a dataloader for the train set
train_dataloader = torch.utils.data.DataLoader(train_set, batch_size=1, shuffle=False)
eval_dataloader = torch.utils.data.DataLoader(eval_set, batch_size=1, shuffle=False)
test_dataloader = torch.utils.data.DataLoader(test_set, batch_size=1, shuffle=False)

# Train

In [30]:
train_x = []
train_y = []
for i, batch in enumerate(train_dataloader):
    inputs,  hard_labels = batch["output"], batch["labels"]
    soft_labels = batch["soft_labels"]["0"][0]#maybe other way around
    train_x.append(inputs)
    train_y.append(soft_labels)

train_x = torch.cat(train_x, dim=0).to(device)
train_y = torch.cat(train_y, dim=0).to(device)


In [31]:
import gpytorch

class GPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(GPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.GridInterpolationKernel(
                gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel(ard_num_dims=768)),
                num_dims=768, grid_size=100
            )
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())
        self.scale_to_bounds = gpytorch.utils.grid.ScaleToBounds(-1., 1.)

    def forward(self, x):
        x = self.scale_to_bounds(x)
        mean = self.mean_module(x)
        covar = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean, covar)

# Define the likelihood function
likelihood = gpytorch.likelihoods.GaussianLikelihood()

# Create a model and move it to the GPU
model = GPModel(train_x, train_y, likelihood)
model.to(device)

GPModel(
  (likelihood): GaussianLikelihood(
    (noise_covar): HomoskedasticNoise(
      (raw_noise_constraint): GreaterThan(1.000E-04)
    )
  )
  (mean_module): ConstantMean()
  (covar_module): ScaleKernel(
    (base_kernel): RBFKernel(
      (raw_lengthscale_constraint): Positive()
    )
    (raw_outputscale_constraint): Positive()
  )
  (scale_to_bounds): ScaleToBounds()
)

In [32]:
# Define the optimizer and train the model
optimizer = torch.optim.Adam([
    {'params': model.covar_module.parameters()},
    {'params': model.mean_module.parameters()},
    {'params': model.likelihood.parameters()},
], lr=0.01)

mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

In [38]:
import tqdm
model.train()

training_iterations = 100

def train():
    iterator = tqdm.notebook.tqdm(range(training_iterations))
    for i in iterator:
        # Zero backprop gradients
        optimizer.zero_grad()
        # Get output from model
        output = model(train_x)
        # Calc loss and backprop derivatives
        loss = -mll(output, train_y)
        loss.backward()
        iterator.set_postfix(loss=loss.item())
        optimizer.step()

%time train()

  0%|          | 0/100 [00:00<?, ?it/s]

CPU times: user 12.7 s, sys: 92.4 ms, total: 12.8 s
Wall time: 13.2 s


In [39]:
test_x = []
test_y = []
for i, batch in enumerate(test_dataloader):
    inputs,  hard_labels = batch["output"], batch["labels"]
    soft_labels = batch["soft_labels"]["0"][0]#maybe other way around
    test_x.append(inputs)
    test_y.append(soft_labels)

test_x = torch.cat(test_x, dim=0).to(device)
test_y = torch.cat(test_y, dim=0).to(device)

In [40]:
model.eval()
likelihood.eval()
with torch.no_grad(), gpytorch.settings.use_toeplitz(False), gpytorch.settings.fast_pred_var():
    preds = model(test_x)

In [54]:
test_y

tensor([1.0000, 1.0000, 0.3330,  ..., 0.8300, 1.0000, 0.6000], device='cuda:0')

In [56]:
preds.variance

tensor([0.1337, 0.1396, 0.1698,  ..., 0.2153, 0.0246, 0.1687], device='cuda:0')

In [53]:
preds.mean

tensor([0.7499, 0.8908, 0.8772,  ..., 0.6311, 0.7996, 0.5067], device='cuda:0')

In [41]:
print('Test MAE: {}'.format(torch.mean(torch.abs(preds.mean - test_y))))

Test MAE: 0.23644913733005524


In [58]:
preds_clipped = torch.clip(preds.mean, 1e-12, 1. - 1e-12)
preds_clipped = torch.clip(preds.mean, 1e-12, 1. - 1e-12)

In [59]:
criterion = torch.nn.BCELoss(reduction="mean")
criterion(preds_clipped, test_y)

tensor(0.5436, device='cuda:0')

# Results

ce 0.5398 , mae 0.23413577675819397, soft_labels = batch["soft_labels"]["0"][0]
1.8794 , .., soft_labels = batch["soft_labels"]["1"][0]

# Alternatives

In [19]:
# import GPy
# import numpy as np

# # Extract the inputs and outputs from the dataset
# X = [batch["output"] for batch in train_set]
# Y = [batch["soft_labels"] for batch in train_set]

# # Convert the inputs and outputs to numpy arrays
# X = np.array(X)
# Y = np.array(Y)

# # Define the kernel for the GP
# kernel = GPy.kern.RBF(input_dim=X.shape[1], ARD=True)

# # Create the GP model
# gp = GPy.models.GPRegression(X, Y, kernel)

# # Optimize the model's hyperparameters
# gp.optimize()


In [20]:
# from tqdm import tqdm_notebook
# from sklearn.model_selection import train_test_split
# import gpflow

# X_train, X_test, y_train, y_test = train_test_split(embeddings, dataset['label'], test_size=0.2, random_state=42)

# # train a Gaussian process model


# kernel = gpflow.kernels.RBF()
# gpmodel = gpflow.models.VGP(X_train, y_train, kern=kernel)

# # optimize the model hyperparameters
# opt = gpflow.optimizers.Scipy()
# opt.minimize(gpmodel.training_loss, gpmodel.trainable_variables)

# # use the trained model to predict the sexual abuse score for the test data with uncertainties
# mean, var = gpmodel.predict_f(X_test)

# # print the predictions and uncertainties
# for i in range(len(mean)):
#     print(f'Prediction: {mean[i]:.2f}, Uncertainty: {var[i]:.2f}')
